# Práctica NASA: 

### Victor Marinas

In [1]:
#Cargamos las correspondientes librerias

from pyspark import SparkContext
sc = SparkContext()

In [13]:
#Utilizamos el archivo pequeño, ya que spark no puede caragar más de 25M

datos = "./apache.access.log_small.txt"
datos_2 = sc.textFile(datos)

In [14]:
#Contamos el numero de registros

datos_2.count()

#Resultado 1043177

3432

In [15]:
#Vemos la estructura del archivo:

datos_2.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [16]:
import re

In [17]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)

In [19]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parseado_rdd = datos_2.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0]) #map con el segundo parseador, filtramos cuales de ellos han sido uno (que el paseador los pasa adecuadamente) y despues mapeamos
parsedado_definitivo = parseado_rdd.map(lambda line: map_log(line))

In [20]:
parsedado_definitivo.take(1)

[('in24.inetnebr.com',
  '-',
  '-',
  '01/Aug/1995:00:00:01',
  '0400',
  'GET',
  '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  'HTTP/1.0"',
  None,
  '200',
  '1839')]

In [21]:
from pyspark.sql import SQLContext, Row #la funcion row, oermite nombrar cada elemento de la lista
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [22]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)

In [25]:
sql_datos = parsedado_definitivo.map(lambda p: Row(
    Host = p[0], 
    date = p[3][:2],
    endpoint = p[6], 
    code = p[9],
    size = convert_long(p[10])))

In [26]:
sql_datos.take(1)

[Row(Host='in24.inetnebr.com', code='200', date='01', endpoint='/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', size=1839)]

In [28]:
dataframe = sqlContext.createDataFrame(sql_datos)
dataframe.registerTempTable("nasa")

In [58]:
#Mostramos la base de datos, la 20 primeras filas

resultado = sqlContext.sql("""
    SELECT * 
    FROM nasa 
    LIMIT 20
""")
resultado.show() 

+--------------------+----+----+--------------------+-----+
|                Host|code|date|            endpoint| size|
+--------------------+----+----+--------------------+-----+
|   in24.inetnebr.com| 200|  01|/shuttle/missions...| 1839|
|     uplherc.upl.com| 304|  01|                   /|    0|
|     uplherc.upl.com| 304|  01|/images/ksclogo-m...|    0|
|     uplherc.upl.com| 304|  01|/images/MOSAIC-lo...|    0|
|     uplherc.upl.com| 304|  01|/images/USA-logos...|    0|
|ix-esc-ca2-07.ix....| 200|  01|/images/launch-lo...| 1713|
|     uplherc.upl.com| 304|  01|/images/WORLD-log...|    0|
|slppp6.intermind.net| 200|  01|/history/skylab/s...| 1687|
|piweba4y.prodigy.com| 200|  01|/images/launchmed...|11853|
|slppp6.intermind.net| 200|  01|/history/skylab/s...| 9202|
|slppp6.intermind.net| 200|  01|/images/ksclogosm...| 3635|
|ix-esc-ca2-07.ix....| 200|  01|/history/apollo/i...| 1173|
|slppp6.intermind.net| 200|  01|/history/apollo/i...| 3047|
|     uplherc.upl.com| 304|  01|/images/

## **1.- Media ,  Máximo y Mínimo del tamaño de las peticiones (size)**

### Media :

In [59]:
media = sqlContext.sql("""
    SELECT ROUND(AVG(size), 0) AS Media  FROM nasa """)


media.show()

+-------+
|  Media|
+-------+
|16052.0|
+-------+



### Máximo :

In [60]:
maximo = sqlContext.sql("""
    SELECT MAX(size) AS Maximo  FROM nasa """)


maximo.show()

+------+
|Maximo|
+------+
|887988|
+------+



### Mínimo :

In [61]:
minimo = sqlContext.sql("""
    SELECT MIN(size) AS Minimo   FROM nasa """)


minimo.show()

+------+
|Minimo|
+------+
|     0|
+------+



## **2.-Número de peticiones de cada código de respuesta (response_code)**

In [57]:
#Seleccionamos el codigo de respuesta, contamos el nº de peticiones(size) de la base de datos que es "nasa" y lo agrupamos

codigo_respuesta = sqlContext.sql("""
    SELECT  code AS Codigo_Respuesta, COUNT(size) AS Numero_Peticiones 
    FROM nasa 
    GROUP BY code
""")
codigo_respuesta.show()

+----------------+-----------------+
|Codigo_Respuesta|Numero_Peticiones|
+----------------+-----------------+
|             200|             3140|
|             302|               50|
|             404|               22|
|             403|                1|
|             304|              219|
+----------------+-----------------+



## **3.-Mostrar 20 host que hayan sido visitados más de de 10 veces.**

In [46]:
# Seleccionamos de la base de datos, la columna host, filtramos por mas de 10 y ordenamos de mayor a menor. Sacamos las 20 primeras


host = sqlContext.sql("""
    SELECT host AS host, size AS Numero_Visitas 
    FROM nasa 
    WHERE size > 10 
    ORDER BY size 
    LIMIT 20
""")
host.show(20)

#Muestra una lista con la siguiente estructura: (host, numero de visitas)

+--------------------+--------------+
|                host|Numero_Visitas|
+--------------------+--------------+
|     uplherc.upl.com|            68|
|bonnou.lab.kdd.co.jp|            68|
|term1-24.sb.west.net|            74|
|   unix.adeptcom.com|            74|
|piweba3y.prodigy.com|            74|
|www-c3.proxy.aol.com|            85|
|line10.pm1.abb.mi...|            85|
|thing1.cchem.berk...|            85|
|ad03-053.compuser...|            85|
|www-c3.proxy.aol.com|            85|
|haraway.ucet.ufl.edu|            91|
|     205.199.120.118|            96|
|www-d4.proxy.aol.com|            96|
|ix-dc7-17.ix.netc...|            96|
|ix-ir8-22.ix.netc...|            96|
|  ncg-69.axionet.com|            96|
|haraway.ucet.ufl.edu|            97|
|term1-24.sb.west.net|            97|
|      205.160.164.19|            98|
|       205.163.36.61|            98|
+--------------------+--------------+



## **4.-Mostrar los 10 endpoints más visitados**

In [47]:
#Seleccionamos los endpoint de la base de datos nasa, ordenamos las visitas de maypr a menor y cogemos las 10 primeras


endpoints10 = sqlContext.sql("""
    SELECT endpoint AS Endpoint, size AS Numero_Visitas 
    FROM nasa 
    ORDER BY size DESC 
    LIMIT 10
""")
endpoints10.show()

#Muestra una lista con la siguiente estructura: (endpoint, numero de visitas)

+--------------------+--------------+
|            Endpoint|Numero_Visitas|
+--------------------+--------------+
|/shuttle/missions...|        887988|
|/shuttle/missions...|        887988|
|/shuttle/missions...|        835394|
|/shuttle/missions...|        806727|
|/shuttle/missions...|        720896|
|/shuttle/missions...|        717587|
|/shuttle/missions...|        717587|
|/shuttle/missions...|        569688|
|     /images/mlp.gif|        517624|
|     /images/mlp.gif|        517624|
+--------------------+--------------+



## **5.-Mostrar los 10 endpoints más visitados, que no tienen código de respuesta =200**

In [48]:
#Seleccionamos los endpoints, cogemos las que no tienen codigo 200, ordenamos de mayor a menor las visitas y cogemos las 10 primeras

endpoints_no200 = sqlContext.sql("""
    SELECT endpoint AS Endpoint, size AS Numero_Visitas
    FROM nasa
    WHERE code != 200
    ORDER BY size DESC
    LIMIT 10
""")
endpoints_no200.show()

#Muestra una lista con la siguiente estructura: (endpoint, numero de visitas)

+--------------------+--------------+
|            Endpoint|Numero_Visitas|
+--------------------+--------------+
|/cgi-bin/imagemap...|           114|
|/cgi-bin/imagemap...|           111|
|/cgi-bin/imagemap...|           111|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
|/cgi-bin/imagemap...|           110|
+--------------------+--------------+



## **6.-Calcular el número de host distintos**

In [35]:
host_distintos = sqlContext.sql("""
    SELECT COUNT(DISTINCT host) AS Hosts
    FROM nasa
""")
host_distintos.show()

+-----+
|Hosts|
+-----+
|  311|
+-----+



## **7.-Contar el número de host únicos cada dia**

In [ ]:
# Ni idea.....

## **8.-Calcular la media de peticiones diarias por host**

In [ ]:
# Ni idea.....

## **9.-Mostrar una lista de 40 endpoints distintos que generan codigo de respuesta = 404**

In [36]:
#Seleccionamos los endpoints, cogemos los distintos a 400, ordenamos de mayor a menor las visitas y cogemos las 40 primeras



endpoints_40 = sqlContext.sql("""
    SELECT DISTINCT endpoint AS Endpoint, code AS Codigo
    FROM nasa
    WHERE code = '404'
    LIMIT 40
""")
endpoints_40.show(40)   #Muestra una lista con la siguiente estructura: (endpoint, Codigo)

+--------------------+------+
|            Endpoint|Codigo|
+--------------------+------+
|/history/apollo/a...|   404|
|/history/apollo/a...|   404|
|/shuttle/resource...|   404|
|/history/apollo/a...|   404|
|/elv/DELTA/uncons...|   404|
|/pub/winvn/readme...|   404|
|/history/apollo/a...|   404|
|/history/history.htm|   404|
|     /sts-71/launch/|   404|
|/history/apollo/a...|   404|
|/www/software/win...|   404|
|/history/apollo/a...|   404|
|/pub/winvn/releas...|   404|
+--------------------+------+



## **10.- Mostrar el top 25 de endpoints  que más codigos de respuesta 404 generan:**

In [37]:

#Seleccionamos los endpoint, contamos los codigos que generan 404, los ordenamos de mayor a menor, seleccionando 25 primeros

resultado = sqlContext.sql("""
    SELECT endpoint AS Endpoint, COUNT(code) AS Codigo
    FROM nasa
    WHERE code = '404'
    GROUP BY endpoint
    ORDER BY COUNT(code) DESC
    LIMIT 25
""")
resultado.show()


+--------------------+------+
|            Endpoint|Codigo|
+--------------------+------+
|/pub/winvn/releas...|     4|
|/history/apollo/a...|     4|
|/pub/winvn/readme...|     2|
|/history/apollo/a...|     2|
|/history/apollo/a...|     2|
|/elv/DELTA/uncons...|     1|
|     /sts-71/launch/|     1|
|/history/apollo/a...|     1|
|/www/software/win...|     1|
|/history/apollo/a...|     1|
|/history/history.htm|     1|
|/history/apollo/a...|     1|
|/shuttle/resource...|     1|
+--------------------+------+



## **11.-El top 5 de dias que se generaron codigo de respuestas 404:**

In [ ]:
# Ni idea.....